# OpenAI API 测试：Web Search Preview 和 Thread 对话功能

本 notebook 用于测试 OpenAI API 的以下功能：
1. Web Search Preview 功能
2. Thread 对话功能（多轮对话）

## 1. 导入所需库

In [1]:
import os
import openai
from openai import OpenAI
import json
import time
from datetime import datetime

## 2. 设置 OpenAI 客户端配置

配置 OpenAI 客户端，需要设置 API key。

In [ ]:
# 设置 API key（建议使用环境变量）
# 方法1：从环境变量获取

# api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("警告：未找到 OPENAI_API_KEY 环境变量，请设置后再运行")
    print("设置方法：export OPENAI_API_KEY='your-api-key'")
else:
    print("API key 已设置")

# 初始化 OpenAI 客户端
client = OpenAI(api_key=api_key)

API key 已设置


In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": "Hello.",
        }
    ],
)

print(completion.choices[0].message.content)

Hi there! How can I assist you today?


In [13]:
completion = client.chat.completions.create(
    model="gpt-4o-search-preview",
    web_search_options={},
    messages=[
        {
            "role": "user",
            # "content": "What is the current weather like in New York?",
            "content": "2025年，美国向巴西征收关税，给我一些相关报道和链接",
        }
    ],
)

print(completion.choices[0].message.content)

2025年7月9日，美国总统唐纳德·特朗普宣布，从8月1日起对所有来自巴西的进口商品征收50%的关税。 ([ft.com](https://www.ft.com/content/c87765ee-94de-4a76-9ed1-0ffa4b94f0fd?utm_source=openai))特朗普表示，此举是对巴西政府对前总统雅伊尔·博索纳罗司法调查的回应，称其为“政治猎巫”，并要求立即终止对博索纳罗的审判。 ([theweek.com](https://theweek.com/politics/trump-tariffs-brazil?utm_source=openai))

巴西总统路易斯·伊纳西奥·卢拉·达席尔瓦对此表示强烈反对，强调巴西是一个独立的主权国家，不会接受任何形式的干预。 ([chinanews.com.cn](https://www.chinanews.com.cn/gj/2025/07-10/10445603.shtml?utm_source=openai))卢拉还表示，巴西将依据《商业互惠法》对美国的关税措施进行对等回应。 ([elpais.com](https://elpais.com/expres/2025-07-11/trump-castiga-con-aranceles-del-50-a-brasil-por-el-juicio-a-bolsonaro-y-lula-da-silva-piensa-responder.html?utm_source=openai))

分析人士指出，美国对巴西征收50%关税可能对巴西的出口经济产生重大影响，特别是咖啡、牛肉、橙汁、石油和航空等关键行业。 ([reuters.com](https://www.reuters.com/business/aerospace-defense/how-50-us-tariff-rate-could-affect-brazilian-exports-2025-07-10/?utm_source=openai))然而，巴西财政部预测，关税对整体经济的影响有限，只有航空航天和能源机械等特定制造业部门会受到显著影响。 ([reuters.com](https://www.reuters.com/world/americas/brazil-downplays-impact-

## 3. 创建 Thread 进行对话

使用 OpenAI 的 thread 功能来维护多轮对话的上下文。

In [24]:
"""
Multi‑turn thread demo for the OpenAI Assistants API.

Usage:
  $ pip install --upgrade openai     # needs openai>=1.59
  $ export OPENAI_API_KEY="sk-..."
  $ python openai_thread_multiturn_demo.py

The script will:
  • create a tiny assistant that remembers prior turns
  • open a fresh thread
  • send two user messages in sequence
  • print the assistant’s replies

The assistant’s second answer should reference the first user question,
confirming that the thread keeps state across turns.
"""


def wait_on_run(thread_id: str, run_id: str):
    """Poll the run until it completes and return the final run object."""
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run.status in {"completed", "failed", "cancelled", "expired"}:
            return run
        time.sleep(0.5)


# 1) Create (or reuse) a minimal assistant ------------------------------------
assistant = client.beta.assistants.create(
    name="Echo & Memory Tester",
    instructions=(
        "You are a super‑concise assistant. "
        "Answer in one sentence and remember earlier messages."
    ),
    model="gpt-4o-mini"  # swap for gpt-4o or gpt-3.5-turbo if needed
)


# 2) Open a brand‑new thread ---------------------------------------------------
thread = client.beta.threads.create()


def chat_once(user_content: str) -> str:
    """Send one user message, run the assistant, and return its reply."""
    # 3) Append the user message to the thread
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_content,
    )

    # 4) Kick off a run for this turn
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # 5) Wait until the run finishes
    run = wait_on_run(thread.id, run.id)

    # 6) Fetch the assistant's newest message
    messages = client.beta.threads.messages.list(thread_id=thread.id, limit=1)
    return messages.data[0].content[0].text.value



In [15]:
# if __name__ == "__main__":
# --- Turn 1 -------------------------------------------------------------
print("\n=== Turn 1 ===")
user_1 = "Hello, what's your name?"
print("User:", user_1)
assistant_1 = chat_once(user_1)
print("Assistant:", assistant_1)

# --- Turn 2 -------------------------------------------------------------
print("\n=== Turn 2 ===")
user_2 = "What did I just ask you?"
print("User:", user_2)
assistant_2 = chat_once(user_2)
print("Assistant:", assistant_2)

# print("\n✔️  Multi‑turn demo complete. If the second assistant reply correctly recalls the first question, your thread is working!")



=== Turn 1 ===
User: Hello, what's your name?
Assistant: I'm your assistant; I don't have a personal name.

=== Turn 2 ===
User: What did I just ask you?
Assistant: You asked for my name.


## 4. 测试web_search


In [25]:
response = client.responses.create(
    model="gpt-4.1",
    tools=[{"type": "web_search_preview"}],
    input="What is the weather of Hangzhou today?"
)

print(response.output_text)

As of 4:32 PM local time on Saturday, July 12, 2025, in Hangzhou, China, the weather is as follows:

## 杭州市, 中华人民共和国 的天气：
当前状况：小雨，85°F (30°C)

每日预报：
* 星期六, 七月 12：低温：75°F (24°C)，高温：87°F (31°C)，描述：多云，潮湿
* 星期日, 七月 13：低温：75°F (24°C)，高温：90°F (32°C)，描述：上午有少量降雨；大部分多云
* 星期一, 七月 14：低温：74°F (23°C)，高温：96°F (35°C)，描述：上午有几场强阵雨；部分晴
* 星期二, 七月 15：低温：78°F (26°C)，高温：97°F (36°C)，描述：暖和，云量增加；下午局部地区有雷雨
* 星期三, 七月 16：低温：80°F (27°C)，高温：101°F (38°C)，描述：部分晴，炎热
* 星期四, 七月 17：低温：79°F (26°C)，高温：98°F (36°C)，描述：多云，暖和潮湿，有几场雷雨
* 星期五, 七月 18：低温：78°F (25°C)，高温：93°F (34°C)，描述：强雷雨


Please note that weather conditions can change rapidly. For the most current information, it's advisable to consult a local weather service. 
